In [2]:
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import *
from sklearn.model_selection import KFold

In [3]:
# Setting Network dan Training
NB_EPOCH = 25
VERBOSE = 1 #Verbosity mode 0 = silent, 1 = progress bar, 2 = one line per epoch.
NB_CLASSES = 7  #jumlah output emosi (kelas target)
N_HIDDEN = 1000 #jumlah hidden layer
N_LINES = 1   #jumlah baris yang mewakili 1 data
N_INPUTFILES = 2800 #jumlah data
BATCH_SIZE = 10 #Number of samples per evaluation step. If unspecified, batch_size will default to 32.

In [4]:
# LOAD MFCC
data = np.genfromtxt("data_mfcc.csv", delimiter=",")
X = [i for i in data[:,0:12]]       #mengambil data pada kolom 0-11
Y = [int(i) for i in data[:,-1]]  #mengambil data pada kolom pertama dari akhir kolom

In [5]:
# Merubah list ke array numpy (list ke data frame)
X = np.array(X)
Y = np.array(Y)

In [6]:
# Normalized X
max = np.amax(X)
min = np.amin(X)
X_normalized = (X-min) / (max-min)

In [7]:
# Convert class vector ke binnary class matrics
# label pada Y = 0: 'angry', 1: 'fear',2: 'surprise',3: 'happy',4: 'sad',5: 'disgust',6: 'neutral'
Y = keras.utils.to_categorical(Y, NB_CLASSES)

In [8]:
# Reshape ke [samples][pixels][width][height]
X = X.reshape(N_INPUTFILES, N_LINES, 12, 1)
X_normalized = X_normalized.reshape(N_INPUTFILES, N_LINES, 12, 1)

In [9]:
# Membuat model CNN
model = Sequential()
model.add(Conv2D(20, kernel_size = 5, padding = "same", input_shape = (N_LINES, 12, 1)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2), strides= (2,2), data_format="channels_first"))
model.add(Flatten())
model.add(Dense(N_HIDDEN))
model.add(Activation("relu"))

In [10]:
# A softmax classifier
model.add(Dense(NB_CLASSES))
model.add(Activation("softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 12, 20)         520       
_________________________________________________________________
activation_1 (Activation)    (None, 1, 12, 20)         0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 6, 10)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 60)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              61000     
_________________________________________________________________
activation_2 (Activation)    (None, 1000)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 7007      
__________

In [11]:
# compile model dengan Adam optimizer
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model_normalize = model

In [12]:
# fold validation k = 10
k_fold = KFold(n_splits=10)
k = 0
sum_score = 0
sum_score_normalized = 0
max_score = 0
max_score_normilazed = 0
best_indices = () #(train, test)
best_indices_normalized = () #(train, test)

In [13]:
for train_indices, test_indices in k_fold.split(X):
    # penambahan k untuk print
    k += 1

    # pengambilan data uji dan data latih
    X_train = X[train_indices]
    X_train_normalized = X_normalized[train_indices]
    Y_train = Y[train_indices]

    X_test = X[test_indices]
    X_test_normalized = X_normalized[test_indices]
    Y_test = Y[test_indices]

    # Fit model
    model_normalize.fit(X_train_normalized, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE)
    model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=VERBOSE)

    # evaluasi
    score = model.evaluate(X_test, Y_test)
    score_normilized = model_normalize.evaluate(X_test_normalized, Y_test)

    if max_score < score[1] * 100:
        max_score = score[1] * 100
        best_indices = (train_indices, test_indices)

    if max_score_normilazed < score_normilized[1] * 100:
        max_score_normilazed = score_normilized[1] * 100
        best_indices_normalized = (train_indices, test_indices)

    print("\n###########################################################################  Fold ke-{}\n".format(k))
    print("Tanpa Normalisasi\n{}: {}".format(model.metrics_names[1], score[1] * 100))
    print("Dengan Normalisasi\n{}: {}".format(model_normalize.metrics_names[1], score_normilized[1] * 100))
    print("\n###########################################################################\n")

    sum_score += score[1] * 100
    sum_score_normalized += score_normilized[1] * 100

avg_score = sum_score/10
avg_score_normalized = sum_score_normalized/10

print("\nRata-rata Akurasi")
print("Tanpa Normalisasi : {}".format(avg_score))
print("Dengan Normalisasi : {}".format(avg_score_normalized))

Epoch 1/25
2520/2520 [==============================] - 2s 659us/step - loss: 0.4072 - acc: 0.8571
Epoch 2/25
2520/2520 [==============================] - 1s 317us/step - loss: 0.3847 - acc: 0.8571
Epoch 3/25
2520/2520 [==============================] - 1s 342us/step - loss: 0.3783 - acc: 0.8571
Epoch 4/25
2520/2520 [==============================] - 1s 318us/step - loss: 0.3733 - acc: 0.8572
Epoch 5/25
2520/2520 [==============================] - 1s 315us/step - loss: 0.3690 - acc: 0.8572
Epoch 6/25
2520/2520 [==============================] - 1s 339us/step - loss: 0.3647 - acc: 0.8577
Epoch 7/25
2520/2520 [==============================] - 1s 328us/step - loss: 0.3583 - acc: 0.8582
Epoch 8/25
2520/2520 [==============================] - 1s 313us/step - loss: 0.3520 - acc: 0.8577
Epoch 9/25
2520/2520 [==============================] - 1s 325us/step - loss: 0.3443 - acc: 0.8584
Epoch 10/25
2520/2520 [==============================] - 1s 321us/step - loss: 0.3346 - acc: 0.8587
Epoch 11/

2520/2520 [==============================] - 1s 307us/step - loss: 3.9258 - acc: 0.7551
Epoch 5/25
2520/2520 [==============================] - 1s 309us/step - loss: 3.9258 - acc: 0.7551
Epoch 6/25
2520/2520 [==============================] - 1s 298us/step - loss: 3.9258 - acc: 0.7551
Epoch 7/25
2520/2520 [==============================] - 1s 316us/step - loss: 3.9258 - acc: 0.7551
Epoch 8/25
2520/2520 [==============================] - 1s 301us/step - loss: 3.9258 - acc: 0.7551
Epoch 9/25
2520/2520 [==============================] - 1s 310us/step - loss: 3.9258 - acc: 0.7551
Epoch 10/25
2520/2520 [==============================] - 1s 304us/step - loss: 3.9258 - acc: 0.7551
Epoch 11/25
2520/2520 [==============================] - 1s 323us/step - loss: 3.9258 - acc: 0.7551
Epoch 12/25
2520/2520 [==============================] - 1s 325us/step - loss: 3.9258 - acc: 0.7551
Epoch 13/25
2520/2520 [==============================] - 1s 318us/step - loss: 3.9258 - acc: 0.7551
Epoch 14/25
2520/

2520/2520 [==============================] - 1s 316us/step - loss: 0.1333 - acc: 0.9471
Epoch 5/25
2520/2520 [==============================] - 1s 300us/step - loss: 0.1331 - acc: 0.9472
Epoch 6/25
2520/2520 [==============================] - 1s 310us/step - loss: 0.1315 - acc: 0.9457
Epoch 7/25
2520/2520 [==============================] - 1s 313us/step - loss: 0.1316 - acc: 0.9457
Epoch 8/25
2520/2520 [==============================] - 1s 312us/step - loss: 0.1296 - acc: 0.9498
Epoch 9/25
2520/2520 [==============================] - 1s 307us/step - loss: 0.1273 - acc: 0.9479
Epoch 10/25
2520/2520 [==============================] - 1s 295us/step - loss: 0.1239 - acc: 0.9500
Epoch 11/25
2520/2520 [==============================] - 1s 296us/step - loss: 0.1251 - acc: 0.9498
Epoch 12/25
2520/2520 [==============================] - 1s 315us/step - loss: 0.1238 - acc: 0.9507
Epoch 13/25
2520/2520 [==============================] - 1s 308us/step - loss: 0.1217 - acc: 0.9510
Epoch 14/25
2520/

2520/2520 [==============================] - 1s 305us/step - loss: 3.9258 - acc: 0.7551
Epoch 8/25
2520/2520 [==============================] - 1s 306us/step - loss: 3.9258 - acc: 0.7551
Epoch 9/25
2520/2520 [==============================] - 1s 320us/step - loss: 3.9258 - acc: 0.7551
Epoch 10/25
2520/2520 [==============================] - 1s 316us/step - loss: 3.9258 - acc: 0.7551
Epoch 11/25
2520/2520 [==============================] - 1s 305us/step - loss: 3.9258 - acc: 0.7551
Epoch 12/25
2520/2520 [==============================] - 1s 309us/step - loss: 3.9258 - acc: 0.7551
Epoch 13/25
2520/2520 [==============================] - 1s 316us/step - loss: 3.9258 - acc: 0.7551
Epoch 14/25
2520/2520 [==============================] - 1s 314us/step - loss: 3.9258 - acc: 0.7551
Epoch 15/25
2520/2520 [==============================] - 1s 308us/step - loss: 3.9258 - acc: 0.7551
Epoch 16/25
2520/2520 [==============================] - 1s 312us/step - loss: 3.9258 - acc: 0.7551
Epoch 17/25
25

2520/2520 [==============================] - 1s 311us/step - loss: 0.0983 - acc: 0.9598
Epoch 8/25
2520/2520 [==============================] - 1s 298us/step - loss: 0.0966 - acc: 0.9603
Epoch 9/25
2520/2520 [==============================] - 1s 314us/step - loss: 0.0924 - acc: 0.9637
Epoch 10/25
2520/2520 [==============================] - 1s 310us/step - loss: 0.0984 - acc: 0.9592
Epoch 11/25
2520/2520 [==============================] - 1s 320us/step - loss: 0.0933 - acc: 0.9620
Epoch 12/25
2520/2520 [==============================] - 1s 309us/step - loss: 0.0901 - acc: 0.9635
Epoch 13/25
2520/2520 [==============================] - 1s 306us/step - loss: 0.0966 - acc: 0.9609
Epoch 14/25
2520/2520 [==============================] - 1s 298us/step - loss: 0.0939 - acc: 0.9618
Epoch 15/25
2520/2520 [==============================] - 1s 302us/step - loss: 0.0965 - acc: 0.9595
Epoch 16/25
2520/2520 [==============================] - 1s 302us/step - loss: 0.0913 - acc: 0.9638
Epoch 17/25
25

2520/2520 [==============================] - 1s 310us/step - loss: 3.9258 - acc: 0.7551
Epoch 11/25
2520/2520 [==============================] - 1s 297us/step - loss: 3.9258 - acc: 0.7551
Epoch 12/25
2520/2520 [==============================] - 1s 306us/step - loss: 3.9258 - acc: 0.7551
Epoch 13/25
2520/2520 [==============================] - 1s 302us/step - loss: 3.9258 - acc: 0.7551
Epoch 14/25
2520/2520 [==============================] - 1s 314us/step - loss: 3.9258 - acc: 0.7551
Epoch 15/25
2520/2520 [==============================] - 1s 305us/step - loss: 3.9258 - acc: 0.7551
Epoch 16/25
2520/2520 [==============================] - 1s 317us/step - loss: 3.9258 - acc: 0.7551
Epoch 17/25
2520/2520 [==============================] - 1s 292us/step - loss: 3.9258 - acc: 0.7551
Epoch 18/25
2520/2520 [==============================] - 1s 313us/step - loss: 3.9258 - acc: 0.7551
Epoch 19/25
2520/2520 [==============================] - 1s 318us/step - loss: 3.9258 - acc: 0.7551
Epoch 20/25


2520/2520 [==============================] - 1s 309us/step - loss: 0.0766 - acc: 0.9694
Epoch 10/25
2520/2520 [==============================] - 1s 310us/step - loss: 0.0738 - acc: 0.9701
Epoch 11/25
2520/2520 [==============================] - 1s 314us/step - loss: 0.0760 - acc: 0.9684
Epoch 12/25
2520/2520 [==============================] - 1s 316us/step - loss: 0.0784 - acc: 0.9679
Epoch 13/25
2520/2520 [==============================] - 1s 309us/step - loss: 0.0757 - acc: 0.9700
Epoch 14/25
2520/2520 [==============================] - 1s 310us/step - loss: 0.0830 - acc: 0.9662
Epoch 15/25
2520/2520 [==============================] - 1s 310us/step - loss: 0.0802 - acc: 0.9668
Epoch 16/25
2520/2520 [==============================] - 1s 314us/step - loss: 0.0743 - acc: 0.9705
Epoch 17/25
2520/2520 [==============================] - 1s 318us/step - loss: 0.0765 - acc: 0.9676
Epoch 18/25
2520/2520 [==============================] - 1s 296us/step - loss: 0.0763 - acc: 0.9698
Epoch 19/25


In [14]:
print("\nAkurasi Terbaik")
if max_score >= max_score_normilazed :
    (train, test) = best_indices
    if max_score == max_score_normilazed :
        print(
            "Tanpa maupun dengan normalisasi memiliki score yang sama yaitu {}, dengan iterasi: \ntrain = {}\ntest = {}".format(
                max_score, train, test))
    else:
        print("Tanpa normalisasi, dengan score {} dan iterasi: \ntrain = {}\ntest = {}".format(max_score, train, test))

    # simpan model
    model.fit(X[train], Y[train], batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=0)
    model.save('model.h5')
    print("Model tersimpan")
else :
    (train, test) = best_indices_normalized
    print("Dengan normalisasi, memiliki score {} dan iterasi: \ntrain = {}\ntest = {}".format(max_score_normilazed, train, test))

    # simpan model
    model_normalize.fit(X_normalized[train], Y[train], batch_size=BATCH_SIZE, epochs=NB_EPOCH, verbose=0)
    model_normalize.save('model.h5')
    print("Model tersimpan")


Akurasi Terbaik
Dengan normalisasi, memiliki score 94.03061475072589 dan iterasi: 
train = [   0    1    2 ... 2797 2798 2799]
test = [1120 1121 1122 1123 1124 1125 1126 1127 1128 1129 1130 1131 1132 1133
 1134 1135 1136 1137 1138 1139 1140 1141 1142 1143 1144 1145 1146 1147
 1148 1149 1150 1151 1152 1153 1154 1155 1156 1157 1158 1159 1160 1161
 1162 1163 1164 1165 1166 1167 1168 1169 1170 1171 1172 1173 1174 1175
 1176 1177 1178 1179 1180 1181 1182 1183 1184 1185 1186 1187 1188 1189
 1190 1191 1192 1193 1194 1195 1196 1197 1198 1199 1200 1201 1202 1203
 1204 1205 1206 1207 1208 1209 1210 1211 1212 1213 1214 1215 1216 1217
 1218 1219 1220 1221 1222 1223 1224 1225 1226 1227 1228 1229 1230 1231
 1232 1233 1234 1235 1236 1237 1238 1239 1240 1241 1242 1243 1244 1245
 1246 1247 1248 1249 1250 1251 1252 1253 1254 1255 1256 1257 1258 1259
 1260 1261 1262 1263 1264 1265 1266 1267 1268 1269 1270 1271 1272 1273
 1274 1275 1276 1277 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287
 1288 1289 12